# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [11]:
import os
import gzip
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim

# Data Management

DATA_DIR = "data/fashion"
TRAIN_IMG = os.path.join(DATA_DIR, "train-images-idx3-ubyte.gz")
TRAIN_LBL = os.path.join(DATA_DIR, "train-labels-idx1-ubyte.gz")

os.makedirs(DATA_DIR, exist_ok=True)

# Downloaded manual files (wget)
os.system(f"wget -O {TRAIN_IMG} http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz")
os.system(f"wget -O {TRAIN_LBL} http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz")

def read_images(file_path):
    with gzip.open(file_path, 'rb') as f:
        f.read(16)
        data = f.read()
        images = np.frombuffer(data, dtype=np.uint8).reshape(-1, 1, 28, 28)
        return images.astype(np.float32) / 255.0

def read_labels(file_path):
    with gzip.open(file_path, 'rb') as f:
        f.read(8)
        return np.frombuffer(f.read(), dtype=np.uint8)

class FashionDataset(Dataset):
    def __init__(self, images_path, labels_path):
        self.images = torch.tensor(read_images(images_path), dtype=torch.float32)
        self.labels = torch.tensor(read_labels(labels_path), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]


#Model Definition

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)


#Training Function

def train_model(model, train_loader, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    model.train()

    for epoch in range(10):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "fashion_model.pth")
    print("✅ Model saved as fashion_model.pth")


#Main Function

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️ Using device: {device}")

    # Load Data
    train_data = FashionDataset(TRAIN_IMG, TRAIN_LBL)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    # Create model
    model = FashionNet().to(device)

    # Train
    train_model(model, train_loader, device)

if __name__ == "__main__":
    main()

🖥️ Using device: cpu
Epoch 1, Loss: 0.5575
Epoch 2, Loss: 0.3893
Epoch 3, Loss: 0.3484
Epoch 4, Loss: 0.3222
Epoch 5, Loss: 0.3066
Epoch 6, Loss: 0.2889
Epoch 7, Loss: 0.2765
Epoch 8, Loss: 0.2645
Epoch 9, Loss: 0.2546
Epoch 10, Loss: 0.2485
✅ Model saved as fashion_model.pth


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import gzip
import numpy as np

# Paths
DATA_DIR = "data/fashion"
TEST_IMG = os.path.join(DATA_DIR, "t10k-images-idx3-ubyte.gz")
TEST_LBL = os.path.join(DATA_DIR, "t10k-labels-idx1-ubyte.gz")

# Download
os.makedirs(DATA_DIR, exist_ok=True)
os.system(f"wget -O {TEST_IMG} http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz")
os.system(f"wget -O {TEST_LBL} http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz")

# Data loading
def read_images(path):
    with gzip.open(path, 'rb') as f:
        f.read(16)
        data = f.read()
        images = np.frombuffer(data, dtype=np.uint8).reshape(-1, 1, 28, 28)
        return images.astype(np.float32) / 255.0

def read_labels(path):
    with gzip.open(path, 'rb') as f:
        f.read(8)
        return np.frombuffer(f.read(), dtype=np.uint8)

class FashionDataset(Dataset):
    def __init__(self, images_path, labels_path):
        self.images = torch.tensor(read_images(images_path), dtype=torch.float32)
        self.labels = torch.tensor(read_labels(labels_path), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Same model structure
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

# Evaluation logic
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"✅ Test Accuracy: {accuracy:.2f}%")

# Main function
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🧪 Evaluating on device: {device}")

    # Load model
    model = FashionNet().to(device)
    model.load_state_dict(torch.load("fashion_model.pth", map_location=device))

    # Load test data
    test_data = FashionDataset(TEST_IMG, TEST_LBL)
    test_loader = DataLoader(test_data, batch_size=64)

    # Evaluate
    evaluate_model(model, test_loader, device)


🧪 Evaluating on device: cpu
✅ Test Accuracy: 88.45%
